## Preprocessing

In [4]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Import our dependencies

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df = application_df.drop(columns=['EIN','NAME'])
df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Determine the number of unique values in each column.
df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

There are three columns that have more than 10 unique values. The categorical types will need to be binned to reduce the number of unique values. Numerical data does not need to be binned.

APPLICATION_TYPE: 17

CLASSIFICATION: 71

ASK_AMT: 8,747

In [4]:
# Look at APPLICATION_TYPE value counts for binning
df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# if type < 100, group as 'other'
application_types_to_replace = ['T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
cls1 = df['CLASSIFICATION'].value_counts()
cls1

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2600        1
C8210        1
C2500        1
C1732        1
C1900        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1

cls = df['CLASSIFICATION'].value_counts().loc[lambda x : x>1] 
print(len(cls))
cls

45


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
C1235        9
C1237        9
C7210        7
C4100        6
C1720        6
C2400        6
C1257        5
C1600        5
C0           3
C1260        3
C2710        3
C3200        2
C1267        2
C1256        2
C1234        2
C1246        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# FILTER CLS by values less than 1000. Use index for list
classifications_to_replace = (cls1[cls1<1000]).index
classifications_to_replace

Index(['C7000', 'C1700', 'C4000', 'C5000', 'C1270', 'C2700', 'C2800', 'C7100',
       'C1300', 'C1280', 'C1230', 'C1400', 'C7200', 'C2300', 'C1240', 'C8000',
       'C7120', 'C1500', 'C1800', 'C6000', 'C1250', 'C8200', 'C1278', 'C1238',
       'C1235', 'C1237', 'C7210', 'C4100', 'C1720', 'C2400', 'C1257', 'C1600',
       'C0', 'C1260', 'C2710', 'C3200', 'C1267', 'C1256', 'C1234', 'C1246',
       'C1245', 'C2170', 'C1728', 'C1248', 'C4200', 'C4120', 'C5200', 'C4500',
       'C1370', 'C1570', 'C1820', 'C2561', 'C3700', 'C2190', 'C6100', 'C1580',
       'C2570', 'C2380', 'C2150', 'C1236', 'C1283', 'C2600', 'C8210', 'C2500',
       'C1732', 'C1900'],
      dtype='object')

In [9]:
# Replace in dataframe
for j in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(j,"Other")
    
# Check to make sure binning was successful
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`

print(df.dtypes)
print('Status:')
print(df['STATUS'].value_counts())
print('INCOME_AMT')
print(df['INCOME_AMT'].value_counts())
print('Is successful:')
print(df['IS_SUCCESSFUL'].value_counts())
df

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object
Status:
1    34294
0        5
Name: STATUS, dtype: int64
INCOME_AMT
0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64
Is successful:
1    18261
0    16038
Name: IS_SUCCESSFUL, dtype: int64


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


The STATUS data is imbalanced - 5 out of 34294 instances are negative. This may be something to consider later when optimizing the model.


In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
df_cat = pd.get_dummies(df)
df_cat

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [12]:
# Split our preprocessed data into our features and target arrays

# The target column is 'IS_SUCCESSFUL'
X = df_cat.drop(['IS_SUCCESSFUL'], axis=1)
y = df_cat['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset

X_train, X_test, y_train, y_test = (train_test_split(X, y, random_state=1))

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# First hidden layer - INPUT FEATURES: 44. 2 neurons per feature: 88
nn_model.add(tf.keras.layers.Dense(units=88, activation="relu", input_dim=44))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 88)                3960      
                                                                 
 dense_1 (Dense)             (None, 30)                2670      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 6,661
Trainable params: 6,661
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the model
# Create a callback that saves the model's weights every five epochs.
model_weights = tf.keras.callbacks.ModelCheckpoint(
    filepath='./model01/checkpoint',
    verbose= 1,
    save_weights_only= True,
    mode='auto',
    save_freq='epoch',
    period=5
)

fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100, callbacks=[model_weights])

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5740 - accuracy: 0.7175
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5553 - accuracy: 0.7278
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5521 - accuracy: 0.7308
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5513 - accuracy: 0.7319
Epoch 5/100
773/804 [===========================>..] - ETA: 0s - loss: 0.5482 - accuracy: 0.7321
Epoch 5: saving model to ./model01\checkpoint
804/804 [==============================] - 2s 2ms/step - loss: 0.5495 - accuracy: 0.7313
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5484 - accuracy: 0.7326
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5480 - accuracy: 0.7336
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5469 - accuracy: 0.7334
Epoch 9/100
804/804 [==============================] - 1s 1ms

804/804 [==============================] - 2s 2ms/step - loss: 0.5363 - accuracy: 0.7390
Epoch 65/100
769/804 [===========================>..] - ETA: 0s - loss: 0.5362 - accuracy: 0.7390
Epoch 65: saving model to ./model01\checkpoint
804/804 [==============================] - 2s 2ms/step - loss: 0.5359 - accuracy: 0.7388
Epoch 66/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5364 - accuracy: 0.7397
Epoch 67/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5360 - accuracy: 0.7393
Epoch 68/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5365 - accuracy: 0.7398
Epoch 69/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5364 - accuracy: 0.7387
Epoch 70/100
804/804 [==============================] - ETA: 0s - loss: 0.5359 - accuracy: 0.7394
Epoch 70: saving model to ./model01\checkpoint
804/804 [==============================] - 1s 2ms/step - loss: 0.5359 - accuracy: 0.7394
Epoch 71/100
804/804 [=============

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Model 1 Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5577 - accuracy: 0.7298 - 387ms/epoch - 1ms/step
Model 1 Loss: 0.5576978325843811, Accuracy: 0.7297959327697754


In [18]:
# Export our model to HDF5 file
nn_model.save('AlphabetSoupCharity.h5')

### Explore Model Optimization ###

### Create Model 2 using best accuracy weights ####

In [19]:
# Train the model
# Create a callback that saves the model's weights every epoch IF it is the best seen so far.
best_model_weights_2 = tf.keras.callbacks.ModelCheckpoint(
    filepath='./model02/checkpoint',
    verbose= 1,
    save_weights_only= True,
    monitor= 'accuracy',
    mode='max',
    save_best_only=True,
    save_freq='epoch'
)

fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100, callbacks=[best_model_weights_2])

Epoch 1/100
783/804 [============================>.] - ETA: 0s - loss: 0.5343 - accuracy: 0.7402
Epoch 1: accuracy improved from -inf to 0.74005, saving model to ./model02\checkpoint
804/804 [==============================] - 1s 2ms/step - loss: 0.5344 - accuracy: 0.7400
Epoch 2/100
785/804 [============================>.] - ETA: 0s - loss: 0.5346 - accuracy: 0.7400
Epoch 2: accuracy did not improve from 0.74005
804/804 [==============================] - 1s 1ms/step - loss: 0.5344 - accuracy: 0.7400
Epoch 3/100
769/804 [===========================>..] - ETA: 0s - loss: 0.5344 - accuracy: 0.7404
Epoch 3: accuracy improved from 0.74005 to 0.74051, saving model to ./model02\checkpoint
804/804 [==============================] - 3s 3ms/step - loss: 0.5345 - accuracy: 0.7405
Epoch 4/100
797/804 [============================>.] - ETA: 0s - loss: 0.5346 - accuracy: 0.7399
Epoch 4: accuracy did not improve from 0.74051
804/804 [==============================] - 1s 1ms/step - loss: 0.5345 - accu

796/804 [============================>.] - ETA: 0s - loss: 0.5331 - accuracy: 0.7413
Epoch 35: accuracy did not improve from 0.74141
804/804 [==============================] - 2s 2ms/step - loss: 0.5334 - accuracy: 0.7411
Epoch 36/100
799/804 [============================>.] - ETA: 0s - loss: 0.5330 - accuracy: 0.7404
Epoch 36: accuracy did not improve from 0.74141
804/804 [==============================] - 2s 3ms/step - loss: 0.5330 - accuracy: 0.7404
Epoch 37/100
804/804 [==============================] - ETA: 0s - loss: 0.5335 - accuracy: 0.7411
Epoch 37: accuracy did not improve from 0.74141
804/804 [==============================] - 1s 1ms/step - loss: 0.5335 - accuracy: 0.7411
Epoch 38/100
794/804 [============================>.] - ETA: 0s - loss: 0.5331 - accuracy: 0.7408
Epoch 38: accuracy did not improve from 0.74141
804/804 [==============================] - 1s 1ms/step - loss: 0.5336 - accuracy: 0.7405
Epoch 39/100
790/804 [============================>.] - ETA: 0s - loss: 0

Epoch 70/100
793/804 [============================>.] - ETA: 0s - loss: 0.5328 - accuracy: 0.7405
Epoch 70: accuracy did not improve from 0.74168
804/804 [==============================] - 2s 2ms/step - loss: 0.5322 - accuracy: 0.7409
Epoch 71/100
777/804 [===========================>..] - ETA: 0s - loss: 0.5328 - accuracy: 0.7408
Epoch 71: accuracy did not improve from 0.74168
804/804 [==============================] - 1s 1ms/step - loss: 0.5326 - accuracy: 0.7414
Epoch 72/100
799/804 [============================>.] - ETA: 0s - loss: 0.5324 - accuracy: 0.7412
Epoch 72: accuracy did not improve from 0.74168
804/804 [==============================] - 2s 3ms/step - loss: 0.5328 - accuracy: 0.7413
Epoch 73/100
803/804 [============================>.] - ETA: 0s - loss: 0.5328 - accuracy: 0.7406
Epoch 73: accuracy did not improve from 0.74168
804/804 [==============================] - 2s 2ms/step - loss: 0.5329 - accuracy: 0.7406
Epoch 74/100
799/804 [============================>.] - ETA:

In [20]:
nn_model.load_weights('./model02/checkpoint')
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Model 2 Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5691 - accuracy: 0.7278 - 544ms/epoch - 2ms/step
Model 2 Loss: 0.5690852403640747, Accuracy: 0.7278134226799011


#### Model 2 outcome ###
There is no improvement with Model 2.

### Model 3 ###
Add third layer, use tanh activation function

In [29]:
# Optimisation: Model 3. 
# Add more layers and tahn to activation function
nn_model = tf.keras.models.Sequential()

# First hidden layer - INPUT FEATURES: 44. 2 neurons per feature: 82
nn_model.add(tf.keras.layers.Dense(units=88, activation="relu", input_dim=44))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=60, activation="tanh"))

# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=30, activation="tanh"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
print('Model 3')
nn_model.summary()

Model 3
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 88)                3960      
                                                                 
 dense_16 (Dense)            (None, 60)                5340      
                                                                 
 dense_17 (Dense)            (None, 30)                1830      
                                                                 
 dense_18 (Dense)            (None, 1)                 31        
                                                                 
Total params: 11,161
Trainable params: 11,161
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [31]:
# Train the model
# Create a callback that saves the model's weights every five epochs.
model_weights_3 = tf.keras.callbacks.ModelCheckpoint(
    filepath='./model03/checkpoint',
    verbose= 1,
    save_weights_only= True,
    mode='auto',
    save_freq='epoch',
    period=5
)

fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100, callbacks=[model_weights_3])

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5684 - accuracy: 0.7203
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5555 - accuracy: 0.7282
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5518 - accuracy: 0.7293
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5505 - accuracy: 0.7306
Epoch 5/100
794/804 [============================>.] - ETA: 0s - loss: 0.5490 - accuracy: 0.7313
Epoch 5: saving model to ./model03\checkpoint
804/804 [==============================] - 2s 3ms/step - loss: 0.5492 - accuracy: 0.7315
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5478 - accuracy: 0.7321
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5473 - accuracy: 0.7344
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5460 - accuracy: 0.7331
Epoch 9/100
804/804 [==============================] - 1s 2ms

804/804 [==============================] - 2s 3ms/step - loss: 0.5349 - accuracy: 0.7401
Epoch 65/100
781/804 [============================>.] - ETA: 0s - loss: 0.5347 - accuracy: 0.7402
Epoch 65: saving model to ./model03\checkpoint
804/804 [==============================] - 1s 2ms/step - loss: 0.5349 - accuracy: 0.7403
Epoch 66/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5350 - accuracy: 0.7399
Epoch 67/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5348 - accuracy: 0.7406
Epoch 68/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5349 - accuracy: 0.7397
Epoch 69/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5347 - accuracy: 0.7407
Epoch 70/100
802/804 [============================>.] - ETA: 0s - loss: 0.5346 - accuracy: 0.7401
Epoch 70: saving model to ./model03\checkpoint
804/804 [==============================] - 3s 4ms/step - loss: 0.5346 - accuracy: 0.7401
Epoch 71/100
804/804 [=============

In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Model 3 Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5556 - accuracy: 0.7291 - 370ms/epoch - 1ms/step
Model 3 Loss: 0.55560702085495, Accuracy: 0.7290962338447571


#### Model 3 outcome ####
There was no improvement with Model 3.

### Model 4 ###
More neurons for first layer, add fourth hidden layer, more relu than tanh activation functions. Increase epochs to 200.

In [33]:
# Optimisation: Model 4.
# Add more neurons, a fourth hidden layer and increase Epochs
nn_model = tf.keras.models.Sequential()

# First hidden layer - INPUT FEATURES: 44. 3 neurons per feature: 132
nn_model.add(tf.keras.layers.Dense(units=132, activation="relu", input_dim=44))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=80, activation="relu"))

# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=60, activation="relu"))

# Fourth hidden layer
nn_model.add(tf.keras.layers.Dense(units=40, activation="tanh"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

print('Model 4 Summary')
# Check the structure of the model
nn_model.summary()

Model 4 Summary
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 132)               5940      
                                                                 
 dense_20 (Dense)            (None, 80)                10640     
                                                                 
 dense_21 (Dense)            (None, 60)                4860      
                                                                 
 dense_22 (Dense)            (None, 40)                2440      
                                                                 
 dense_23 (Dense)            (None, 1)                 41        
                                                                 
Total params: 23,921
Trainable params: 23,921
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [35]:
# Train the model
# Create a callback that saves the model's weights every five epochs.
model_weights_4 = tf.keras.callbacks.ModelCheckpoint(
    filepath='./model04/checkpoint',
    verbose= 1,
    save_weights_only= True,
    mode='auto',
    save_freq='epoch',
    period=5
)

fit_model = nn_model.fit(X_train_scaled, y_train, epochs=200, callbacks=[model_weights_4])

Epoch 1/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5661 - accuracy: 0.7237
Epoch 2/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5545 - accuracy: 0.7294
Epoch 3/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5527 - accuracy: 0.7303
Epoch 4/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5502 - accuracy: 0.7312
Epoch 5/200
775/804 [===========================>..] - ETA: 0s - loss: 0.5490 - accuracy: 0.7316
Epoch 5: saving model to ./model04\checkpoint
804/804 [==============================] - 2s 3ms/step - loss: 0.5492 - accuracy: 0.7315
Epoch 6/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5482 - accuracy: 0.7323
Epoch 7/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5469 - accuracy: 0.7346
Epoch 8/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5472 - accuracy: 0.7338
Epoch 9/200
804/804 [==============================] - 1s 2ms

804/804 [==============================] - 3s 3ms/step - loss: 0.5345 - accuracy: 0.7391
Epoch 65/200
790/804 [============================>.] - ETA: 0s - loss: 0.5354 - accuracy: 0.7383
Epoch 65: saving model to ./model04\checkpoint
804/804 [==============================] - 2s 2ms/step - loss: 0.5350 - accuracy: 0.7385
Epoch 66/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5346 - accuracy: 0.7395
Epoch 67/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5349 - accuracy: 0.7395
Epoch 68/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5353 - accuracy: 0.7389
Epoch 69/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5350 - accuracy: 0.7385
Epoch 70/200
799/804 [============================>.] - ETA: 0s - loss: 0.5341 - accuracy: 0.7403
Epoch 70: saving model to ./model04\checkpoint
804/804 [==============================] - 3s 4ms/step - loss: 0.5344 - accuracy: 0.7399
Epoch 71/200
804/804 [=============

804/804 [==============================] - 2s 3ms/step - loss: 0.5335 - accuracy: 0.7404
Epoch 128/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5332 - accuracy: 0.7413
Epoch 129/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5331 - accuracy: 0.7406
Epoch 130/200
784/804 [============================>.] - ETA: 0s - loss: 0.5318 - accuracy: 0.7421
Epoch 130: saving model to ./model04\checkpoint
804/804 [==============================] - 2s 3ms/step - loss: 0.5324 - accuracy: 0.7415
Epoch 131/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5345 - accuracy: 0.7400
Epoch 132/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5332 - accuracy: 0.7406
Epoch 133/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5325 - accuracy: 0.7403
Epoch 134/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5324 - accuracy: 0.7405
Epoch 135/200
792/804 [============================>.] - 

804/804 [==============================] - 2s 2ms/step - loss: 0.5340 - accuracy: 0.7397
Epoch 191/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5334 - accuracy: 0.7402
Epoch 192/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5328 - accuracy: 0.7412
Epoch 193/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5327 - accuracy: 0.7414
Epoch 194/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5342 - accuracy: 0.7406
Epoch 195/200
800/804 [============================>.] - ETA: 0s - loss: 0.5346 - accuracy: 0.7406
Epoch 195: saving model to ./model04\checkpoint
804/804 [==============================] - 2s 2ms/step - loss: 0.5344 - accuracy: 0.7407
Epoch 196/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5338 - accuracy: 0.7405
Epoch 197/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5337 - accuracy: 0.7405
Epoch 198/200
804/804 [==============================] - 

In [36]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Model 4 Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5623 - accuracy: 0.7287 - 400ms/epoch - 1ms/step
Model 4 Loss: 0.5622939467430115, Accuracy: 0.7287463545799255


#### Model 4 outcome ####
There was no improvement with Model 4.

### Model 5 ###
Add a fifth hidden layer, include sigmoid, tanh and relu activation functions, 200 epochs.

In [37]:
# Optimisation: Model 5. 
# Add more layers and tahn to activation function. 200 Epochs.
nn_model = tf.keras.models.Sequential()

# First hidden layer - INPUT FEATURES: 44. 3 neurons per feature: 132
nn_model.add(tf.keras.layers.Dense(units=132, activation="sigmoid", input_dim=44))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=62, activation="tanh"))

# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=53, activation="relu"))

# Fourth hidden layer
nn_model.add(tf.keras.layers.Dense(units=30, activation="tanh"))

# Fifth hidden layer
nn_model.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 132)               5940      
                                                                 
 dense_25 (Dense)            (None, 62)                8246      
                                                                 
 dense_26 (Dense)            (None, 53)                3339      
                                                                 
 dense_27 (Dense)            (None, 30)                1620      
                                                                 
 dense_28 (Dense)            (None, 30)                930       
                                                                 
 dense_29 (Dense)            (None, 1)                 31        
                                                                 
Total params: 20,106
Trainable params: 20,106
Non-trai

In [38]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [39]:
# Train the model
# Create a callback that saves the model's weights every five epochs.
model_weights_5 = tf.keras.callbacks.ModelCheckpoint(
    filepath='./model05/checkpoint',
    verbose= 1,
    save_weights_only= True,
    mode='auto',
    save_freq='epoch',
    period=5
)

fit_model = nn_model.fit(X_train_scaled, y_train, epochs=200, callbacks=[model_weights_5])

Epoch 1/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5823 - accuracy: 0.7138
Epoch 2/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5622 - accuracy: 0.7264
Epoch 3/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5584 - accuracy: 0.7288
Epoch 4/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5576 - accuracy: 0.7283
Epoch 5/200
783/804 [============================>.] - ETA: 0s - loss: 0.5558 - accuracy: 0.7294
Epoch 5: saving model to ./model05\checkpoint
804/804 [==============================] - 4s 5ms/step - loss: 0.5551 - accuracy: 0.7300
Epoch 6/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5536 - accuracy: 0.7303
Epoch 7/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5524 - accuracy: 0.7325
Epoch 8/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5523 - accuracy: 0.7306
Epoch 9/200
804/804 [==============================] - 2s 2ms

804/804 [==============================] - 1s 2ms/step - loss: 0.5367 - accuracy: 0.7392
Epoch 65/200
796/804 [============================>.] - ETA: 0s - loss: 0.5359 - accuracy: 0.7396
Epoch 65: saving model to ./model05\checkpoint
804/804 [==============================] - 3s 3ms/step - loss: 0.5355 - accuracy: 0.7399
Epoch 66/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5359 - accuracy: 0.7399
Epoch 67/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5355 - accuracy: 0.7388
Epoch 68/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5355 - accuracy: 0.7399
Epoch 69/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5358 - accuracy: 0.7395
Epoch 70/200
802/804 [============================>.] - ETA: 0s - loss: 0.5353 - accuracy: 0.7391
Epoch 70: saving model to ./model05\checkpoint
804/804 [==============================] - 3s 3ms/step - loss: 0.5352 - accuracy: 0.7393
Epoch 71/200
804/804 [=============

804/804 [==============================] - 2s 2ms/step - loss: 0.5323 - accuracy: 0.7411
Epoch 128/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5319 - accuracy: 0.7407
Epoch 129/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5330 - accuracy: 0.7411
Epoch 130/200
791/804 [============================>.] - ETA: 0s - loss: 0.5318 - accuracy: 0.7404
Epoch 130: saving model to ./model05\checkpoint
804/804 [==============================] - 3s 3ms/step - loss: 0.5320 - accuracy: 0.7402
Epoch 131/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5318 - accuracy: 0.7408
Epoch 132/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5320 - accuracy: 0.7402
Epoch 133/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5314 - accuracy: 0.7414
Epoch 134/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5320 - accuracy: 0.7414
Epoch 135/200
803/804 [============================>.] - 

804/804 [==============================] - 3s 3ms/step - loss: 0.5316 - accuracy: 0.7416
Epoch 191/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5318 - accuracy: 0.7414
Epoch 192/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5314 - accuracy: 0.7412
Epoch 193/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5309 - accuracy: 0.7409
Epoch 194/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5308 - accuracy: 0.7418
Epoch 195/200
791/804 [============================>.] - ETA: 0s - loss: 0.5316 - accuracy: 0.7412
Epoch 195: saving model to ./model05\checkpoint
804/804 [==============================] - 3s 3ms/step - loss: 0.5311 - accuracy: 0.7415
Epoch 196/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5315 - accuracy: 0.7413
Epoch 197/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5312 - accuracy: 0.7414
Epoch 198/200
804/804 [==============================] - 

In [40]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Model 5 Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5697 - accuracy: 0.7272 - 409ms/epoch - 2ms/step
Model 5 Loss: 0.5697187781333923, Accuracy: 0.7272303104400635


#### Model 5 outcome ####
There was no improvement with Model 5.